In [1]:
import os

os.environ["OPENAI_API_KEY"]='<Your Key>'

# Zero-shot Learning

In [2]:
from openai import OpenAI
tool = """
1 tool: python_interpreter, description: use it to execute python code
2 tool: web_access, description: use it to get realtime info, input is the question or query 
"""

react_prompt = f"""
Try your best to anwser user's question, and use the following format:

Question: the input question you must answer

Thought: you should always think about what to do

Action: the action to take, should use one of tools in the given tool list:

[{tool}]

Action Input: the input to the action

Here, you should pause the process and return to wait the outside observation. 

Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)

Thought: I now know the final answer

Final Answer: the final answer to the original input question
"""

def react_demo(request):
    client = OpenAI(
        api_key=os.getenv("DEEPSEEK_API_KEY"),
        base_url="https://api.deepseek.com")
    
    response = client.chat.completions.create(
        model="deepseek-chat",
        temperature = 0,
        messages=[
            {"role": "system", "content": react_prompt,
            },
            {"role": "user", "content": request}
        ]
      )
    print(response.choices[0].message.content)

In [3]:
print(react_demo("今天广州适合穿什么？"))

Question: 今天广州适合穿什么？

Thought: 我需要获取今天广州的天气情况，以便给出合适的穿衣建议。

Action: web_access

Action Input: 今天广州的天气情况
None


In [5]:
# run the web_access tool to get the Guangzhou's weather today. For example, 
weather = "The weather today in Guangzhou is 15°C, with 0% precipitation, 31% humidity, and a wind speed of 4 mph."

next_prompt = """
Question: 今天广州适合穿什么？

Thought: I need to check the current weather in Guangzhou to give a proper clothing suggestion.

Action: web_access

Action Input: 今天广州天气如何？

Observation: """ + weather

print(react_demo(next_prompt))

Thought: Based on the current weather conditions in Guangzhou, I can now suggest appropriate clothing.

Final Answer: 今天广州的天气是15°C，没有降水，湿度31%，风速4 mph。建议穿长袖衬衫或薄毛衣搭配长裤，如果怕冷可以加一件薄外套。
None
